In [1]:
import os
os.getcwd()

In [4]:
# Tester la lecture de ton Excel
import pandas as pd
df = pd.read_excel('excel_base.xlsx')
df.head()  # Voir les premières lignes
df.columns  # Voir les colonnes

Index(['Chambre', 'Emplacement', 'RawScan', 'Nb caisse', 'Nb bocaux',
       'RawScan-Mani p', 'Strain', 'Line', 'Date', 'NbSem', 'AgeAMS', 'Type',
       'Bocaux', 'Milieu', 'Rang', 'XorEorRori ', 'Rang/Rang+', 'Type+Rang',
       'nom_varietes', 'Batch#', 'BatchLines', 'Qualité CHF', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [8]:
import sqlite3
import pandas as pd
from datetime import datetime
import os

class PlantDatabase:
    """Gestion de la base de données SQLite pour le labo de plants in vitro"""
    
    def __init__(self, db_path="plants_lab.db"):
        self.db_path = db_path
        self.conn = None
        
    def connect(self):
        """Ouvre la connexion à la base"""
        self.conn = sqlite3.connect(self.db_path)
        return self.conn
    
    def close(self):
        """Ferme la connexion"""
        if self.conn:
            self.conn.close()
    
    def create_tables(self):
        """Crée la table principale plants"""
        self.connect()
        cursor = self.conn.cursor()
        
        # Table principale : une ligne = une série de plants à un moment donné
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS plants (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                chambre TEXT,
                emplacement TEXT,
                raw_scan TEXT,
                nb_caisse INTEGER,
                nb_bocaux INTEGER,
                raw_scan_mani_p TEXT,
                strain TEXT,
                line INTEGER,
                date TEXT,
                nb_sem INTEGER,
                age_ams TEXT,
                type TEXT,
                bocaux INTEGER,
                milieu TEXT,
                rang INTEGER,
                x_or_e_or_r_or_i TEXT,
                rang_rang_plus TEXT,
                type_rang TEXT,
                nom_varietes TEXT,
                batch_number TEXT,
                batch_lines TEXT,
                qualite_chf TEXT,
                col_22 TEXT,
                col_23 TEXT,
                notes TEXT,
                import_date TEXT,
                is_active INTEGER DEFAULT 1
            )
        """)
        
        self.conn.commit()
        print("✅ Table 'plants' créée avec succès")
        self.close()
    
    def import_from_csv(self, csv_path):
        """Importe les données depuis le CSV"""
        
        if not os.path.exists(csv_path):
            print(f"❌ Erreur : le fichier {csv_path} n'existe pas")
            return
        
        print(f"📂 Lecture du fichier {csv_path}...")
        
        # Lire le CSV avec pandas
        df = pd.read_csv(csv_path, encoding='utf-8')
        
        # Afficher les infos de base
        print(f"📊 {len(df)} lignes trouvées")
        print(f"📊 {len(df.columns)} colonnes trouvées")
        
        # Nettoyer les données
        print("🧹 Nettoyage des données...")
        
        # Supprimer les colonnes vides (Unnamed...)
        cols_to_drop = [col for col in df.columns if 'Unnamed' in str(col) or col in ['0', '1']]
        df = df.drop(columns=cols_to_drop, errors='ignore')
        
        # Renommer les colonnes pour correspondre à la DB
        column_mapping = {
            'Chambre': 'chambre',
            'Emplacement': 'emplacement',
            'RawScan': 'raw_scan',
            'Nb caisse': 'nb_caisse',
            'Nb bocaux': 'nb_bocaux',
            'RawScan-Mani p': 'raw_scan_mani_p',
            'Strain': 'strain',
            'Line': 'line',
            'Date': 'date',
            'NbSem': 'nb_sem',
            'AgeAMS': 'age_ams',
            'Type': 'type',
            'Bocaux': 'bocaux',
            'Milieu': 'milieu',
            'Rang': 'rang',
            'XorEorRori': 'x_or_e_or_r_or_i',
            'Rang/Rang+': 'rang_rang_plus',
            'Type+Rang': 'type_rang',
            'nom_varietes': 'nom_varietes',
            'Batch#': 'batch_number',
            'BatchLines': 'batch_lines',
            'Qualité CHF': 'qualite_chf',
            '< alt+e': 'notes'
        }
        
        df = df.rename(columns=column_mapping)
        
        # Gérer les colonnes 22 et 23 (si elles existent)
        if len([c for c in df.columns if c not in column_mapping.values()]) >= 2:
            other_cols = [c for c in df.columns if c not in column_mapping.values()]
            if len(other_cols) >= 1:
                df = df.rename(columns={other_cols[0]: 'col_22'})
            if len(other_cols) >= 2:
                df = df.rename(columns={other_cols[1]: 'col_23'})
        
        # Convertir les dates au format ISO (YYYY-MM-DD)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        
        # Remplacer les NaN par None (NULL en SQL)
        df = df.where(pd.notna(df), None)
        
        # Ajouter la date d'import
        df['import_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        df['is_active'] = 1
        
        # Importer dans SQLite
        print("💾 Import dans la base de données...")
        self.connect()
        
        # Utiliser to_sql de pandas (super pratique !)
        df.to_sql('plants', self.conn, if_exists='append', index=False)
        
        self.conn.commit()
        self.close()
        
        print(f"✅ {len(df)} lignes importées avec succès dans la table 'plants'")
        print(f"📊 Base de données sauvegardée : {self.db_path}")
    
    def get_stats(self):
        """Affiche des statistiques sur la base"""
        self.connect()
        cursor = self.conn.cursor()
        
        # Nombre total de lignes
        cursor.execute("SELECT COUNT(*) FROM plants WHERE is_active = 1")
        total = cursor.fetchone()[0]
        print(f"\n📊 STATISTIQUES")
        print(f"   Total de séries actives : {total}")
        
        # Par chambre
        cursor.execute("""
            SELECT chambre, COUNT(*) as nb 
            FROM plants 
            WHERE is_active = 1 
            GROUP BY chambre 
            ORDER BY nb DESC
        """)
        print(f"\n   Par chambre :")
        for row in cursor.fetchall():
            print(f"      {row[0]}: {row[1]} séries")
        
        # Par souche
        cursor.execute("""
            SELECT strain, COUNT(*) as nb 
            FROM plants 
            WHERE is_active = 1 
            GROUP BY strain 
            ORDER BY nb DESC 
            LIMIT 10
        """)
        print(f"\n   Top 10 souches :")
        for row in cursor.fetchall():
            print(f"      {row[0]}: {row[1]} séries")
        
        # Par type
        cursor.execute("""
            SELECT type, COUNT(*) as nb 
            FROM plants 
            WHERE is_active = 1 
            GROUP BY type 
            ORDER BY nb DESC
        """)
        print(f"\n   Par type :")
        for row in cursor.fetchall():
            print(f"      {row[0]}: {row[1]} séries")
        
        self.close()
    
    def search_by_barcode(self, barcode):
        """Cherche une série par son code-barres"""
        self.connect()
        cursor = self.conn.cursor()
        
        cursor.execute("""
            SELECT * FROM plants 
            WHERE raw_scan LIKE ? OR raw_scan_mani_p LIKE ?
            AND is_active = 1
        """, (f"%{barcode}%", f"%{barcode}%"))
        
        results = cursor.fetchall()
        self.close()
        
        return results

ImportError: cannot import name 'PlantDatabase' from 'database' (C:\Users\wassi\Marionnet code\database.py)

In [9]:
# Cellule 1 : Imports
import sys
sys.path.append('..')  # Pour pouvoir importer database.py

from database import PlantDatabase
import pandas as pd
import sqlite3

ImportError: cannot import name 'PlantDatabase' from 'database' (C:\Users\wassi\Marionnet code\database.py)